In [82]:
import pandas as pd
import numpy as np
import os

In [83]:
def tranform_icd10(icd10):
    list_disease = ["I10", "I11","I12", "I13", "I14", "I15"]
    if icd10 in list_disease:
        return 1
    else:
        return 0

In [84]:
df = pd.read_csv('./dataset/BPDataset.csv')
df

,icd10,occupation,sex,age,bt,rr,height,weight,lbpn,hbpn,pr,Outcome
0,166,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
1,B009/2,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
2,L810/2,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
3,Z719,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
4,B07,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
65530,R443,902.0,1,14,36.8,20,158.5,48.1,74.0,105,101.0,0
65531,E119,903.0,1,71,36.6,20,155.0,48.1,60.0,113,82.0,0
65532,A090,403.0,1,50,36.4,20,156.0,48.1,61.0,109,81.0,0
65533,D561,719.0,0,42,36.0,20,169.0,48.1,59.0,99,82.0,0


In [4]:
df = df.dropna()
df = df.replace(np.nan, 0)
df["icd10"] = df["icd10"].apply(tranform_icd10)
df

,icd10,occupation,sex,age,bt,rr,height,weight,lbpn,hbpn,pr,Outcome
0,0,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
1,0,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
2,0,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
3,0,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
4,0,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
65530,0,902.0,1,14,36.8,20,158.5,48.1,74.0,105,101.0,0
65531,0,903.0,1,71,36.6,20,155.0,48.1,60.0,113,82.0,0
65532,0,403.0,1,50,36.4,20,156.0,48.1,61.0,109,81.0,0
65533,0,719.0,0,42,36.0,20,169.0,48.1,59.0,99,82.0,0


In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
from sklearn.model_selection import train_test_split

In [17]:
y = df[['icd10']]
x = df.drop('icd10', axis=1)

In [85]:
x

,occupation,sex,age,bt,rr,height,weight,lbpn,hbpn,pr,Outcome
0,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
1,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
2,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
3,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
4,903.0,1,50,37.0,18,100.0,1.0,75.0,110,75.0,0
...,...,...,...,...,...,...,...,...,...,...,...
65530,902.0,1,14,36.8,20,158.5,48.1,74.0,105,101.0,0
65531,903.0,1,71,36.6,20,155.0,48.1,60.0,113,82.0,0
65532,403.0,1,50,36.4,20,156.0,48.1,61.0,109,81.0,0
65533,719.0,0,42,36.0,20,169.0,48.1,59.0,99,82.0,0


In [86]:
y

,icd10
0,0
1,0
2,0
3,0
4,0
...,...
65530,0
65531,0
65532,0
65533,0


In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [38]:
x_train

,occupation,sex,age,bt,rr,height,weight,lbpn,hbpn,pr,Outcome
46678,719.0,1,66,36.4,20,151.0,45.8,57.0,119,86.0,0
26180,99.0,1,65,36.8,20,148.6,42.6,63.0,113,73.0,0
38836,903.0,1,52,36.0,20,153.0,45.0,69.0,108,71.0,0
35119,719.0,1,56,36.2,20,158.0,44.1,72.0,109,62.0,0
28313,403.0,1,56,36.2,20,148.0,43.0,79.0,120,66.0,0
...,...,...,...,...,...,...,...,...,...,...,...
32904,403.0,1,49,36.5,20,160.0,44.0,43.0,103,88.0,0
44154,208.0,0,75,36.4,20,163.0,45.4,60.0,116,82.0,0
30048,208.0,1,76,36.3,20,146.5,43.3,56.0,91,63.0,0
1614,719.0,1,63,36.0,20,153.0,32.0,60.0,104,104.0,0


In [39]:
y_train

,icd10
46678,0
26180,0
38836,0
35119,0
28313,0
...,...
32904,0
44154,0
30048,0
1614,0


In [40]:
x_test

,occupation,sex,age,bt,rr,height,weight,lbpn,hbpn,pr,Outcome
16887,403.0,1,24,36.3,20,166.0,40.1,52.0,93,91.0,0
58500,903.0,1,44,36.9,20,158.0,47.5,65.0,95,95.0,0
5936,403.0,1,69,36.2,20,153.9,36.2,65.0,113,72.0,0
59777,719.0,1,66,35.6,18,149.0,47.6,59.0,120,65.0,0
8772,903.0,1,54,36.8,20,143.0,37.8,65.0,107,98.0,0
...,...,...,...,...,...,...,...,...,...,...,...
48218,201.0,1,57,36.1,20,157.0,46.0,71.0,120,101.0,0
17964,201.0,1,67,36.0,20,150.5,40.5,67.0,116,71.0,0
61475,719.0,1,58,36.6,20,149.5,48.0,65.0,106,92.0,0
8501,403.0,1,61,36.8,20,160.5,37.6,76.0,110,98.0,0


In [41]:
y_test

,icd10
16887,0
58500,0
5936,0
59777,0
8772,0
...,...
48218,0
17964,0
61475,0
8501,0


In [43]:
from sklearn.preprocessing import StandardScaler

In [44]:
scaler = StandardScaler()

In [46]:
x_train = scaler.fit_transform(x_train)
x_train

array([[ 0.63110015,  0.43125542,  0.53764314, ...,  1.05613184,
        -0.0772637 , -0.175912  ],
       [-1.76303456,  0.43125542,  0.48023905, ...,  0.51621613,
        -0.90898792, -0.175912  ],
       [ 1.34161755,  0.43125542, -0.26601406, ...,  0.06628637,
        -1.03694549, -0.175912  ],
       ...,
       [-1.34213023,  0.43125542,  1.11168399, ..., -1.4634748 ,
        -1.54877578, -0.175912  ],
       [ 0.63110015,  0.43125542,  0.36543088, ..., -0.29365743,
         1.07435445, -0.175912  ],
       [ 1.34161755,  0.43125542,  0.07841045, ..., -0.11368553,
        -0.46113642, -0.175912  ]])

In [47]:
x_test = scaler.transform(x_test)
x_test

array([[-0.58913625,  0.43125542, -1.87332846, ..., -1.2835029 ,
         0.24263023, -0.175912  ],
       [ 1.34161755,  0.43125542, -0.72524675, ..., -1.10353099,
         0.49854537, -0.175912  ],
       [-0.58913625,  0.43125542,  0.70985539, ...,  0.51621613,
        -0.97296671, -0.175912  ],
       ...,
       [ 0.63110015,  0.43125542,  0.07841045, ..., -0.11368553,
         0.30660902, -0.175912  ],
       [-0.58913625,  0.43125542,  0.25062271, ...,  0.24625828,
         0.69048173, -0.175912  ],
       [-0.58913625,  0.43125542, -0.20860998, ..., -1.2835029 ,
        -0.39715763, -0.175912  ]])

In [48]:
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
clf.score(x_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.996338996763754

In [53]:
x_pred_sc = scaler.transform(x_test)
y_pred = clf.predict(x_pred_sc)

/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [56]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score

In [87]:
print('Accuarcy:', '{:.4f}'.format(accuracy_score(y_test,y_pred)))

Accuarcy: 0.9966


In [59]:
print('Precision:', '{:.2f}'.format(precision_score(y_test,y_pred)))

Precision: 0.89


In [60]:
print('recall_score:', '{:.2f}'.format(recall_score(y_test,y_pred)))

recall_score: 1.00


In [61]:
print('f1_score:', '{:.2f}'.format(f1_score(y_test,y_pred)))

f1_score: 0.94


In [70]:
cfmx = confusion_matrix(y_test,y_pred)

In [71]:
cfmx

array([[11963,    42],
       [    0,   356]])

In [80]:
err = (cfmx[0,1] + cfmx[1,0]) / int(y_test.count())

In [81]:
print('error rate:', '{:.2f}'.format(err))

error rate: 0.00
